## EODAG CLI

In [1]:
import json
import pathlib

### (Optional) Selection of providers

In [2]:
# import os
# os.environ["EODAG_PROVIDERS_WHITELIST"] = "cop_dataspace"

### List available products

**Note**: use grep to remove detailed information abut the products.

Relevant product for Copernicus DEM are:

* COP_DEM_GLO30_DGED
* COP_DEM_GLO30_DTED
* COP_DEM_GLO90_DGED
* COP_DEM_GLO90_DTED
* cop-dem-glo-30
* cop-dem-glo-90

Relevant product for Sentinel-1 are:

* S1_SAR_GRD
* S1_SAR_GRD_COG
* S1_SAR_OCN
* S1_SAR_RAW
* S1_SAR_SLC
* sentinel-1-rtc

In [3]:
%%bash
eodag list | grep '^* '

* 3dep-lidar-classification: 
* 3dep-lidar-copc: 
* 3dep-lidar-dsm: 
* 3dep-lidar-dtm: 
* 3dep-lidar-dtm-native: 
* 3dep-lidar-hag: 
* 3dep-lidar-intensity: 
* 3dep-lidar-pointsourceid: 
* 3dep-lidar-returns: 
* 3dep-seamless: 
* ANTARCTIC_OMI_SI_extent: 
* ANTARCTIC_OMI_SI_extent_obs: 
* ARCTIC_ANALYSISFORECAST_BGC_002_004: 
* ARCTIC_ANALYSISFORECAST_PHY_002_001: 
* ARCTIC_ANALYSISFORECAST_PHY_ICE_002_011: 
* ARCTIC_ANALYSISFORECAST_PHY_TIDE_002_015: 
* ARCTIC_ANALYSIS_FORECAST_WAV_002_014: 
* ARCTIC_MULTIYEAR_BGC_002_005: 
* ARCTIC_MULTIYEAR_PHY_002_003: 
* ARCTIC_MULTIYEAR_PHY_ICE_002_016: 
* ARCTIC_MULTIYEAR_WAV_002_013: 
* ARCTIC_OMI_SI_Transport_NordicSeas: 
* ARCTIC_OMI_SI_extent: 
* ARCTIC_OMI_SI_extent_obs: 
* ARCTIC_OMI_TEMPSAL_FWC: 
* BALTICSEA_ANALYSISFORECAST_BGC_003_007: 
* BALTICSEA_ANALYSISFORECAST_PHY_003_006: 
* BALTICSEA_ANALYSISFORECAST_WAV_003_010: 
* BALTICSEA_MULTIYEAR_BGC_003_012: 
* BALTICSEA_MULTIYEAR_PHY_003_011: 
* BALTICSEA_MULTIYEAR_WAV_003_015: 
* BALTIC_

### Search DGED-90 DEM tiles intersecting the specified polygon

In [4]:
polygon_wkt = (
    "POLYGON ((111.623686 -30.422697, 112.870434 -26.377362, 113.759005"
    " -25.655883, 114.765229 -25.708236, 115.744513 -26.528056, 116.666913"
    " -26.189345, 117.624098 -26.513583, 118.168346 -27.956584, 116.881996"
    " -32.007367, 115.588516 -32.562306, 114.147425 -31.54362, 113.096861"
    " -31.948575, 112.15162 -31.591408, 111.623686 -30.422697))"
)

**Note**: the `--items` option has been set to `50` to have all the items
in the `search_results.geojson` file.
By default only `20` items are returned.

In [5]:
!eodag search --productType COP_DEM_GLO90_DGED --count --geom "{polygon_wkt}" \
    --items 50

Found a total number of 33 products
Returned 33 products
Results stored at 'search_results.geojson'


Load teh `sarch_results.geojson` file and display the product IDs.

In [6]:
search_results = json.loads(pathlib.Path("search_results.geojson").read_text())

In [7]:
[item["id"] for item in search_results["features"]]

['DEM1_SAR_DGE_90_20101223T213307_20121018T214419_ADS_000000_9271',
 'DEM1_SAR_DGE_90_20101223T213307_20121029T214521_ADS_000000_8099',
 'DEM1_SAR_DGE_90_20101223T213307_20121120T214521_ADS_000000_0831',
 'DEM1_SAR_DGE_90_20101223T213307_20121120T214521_ADS_000000_0870',
 'DEM1_SAR_DGE_90_20101223T213307_20121120T214521_ADS_000000_2367',
 'DEM1_SAR_DGE_90_20101223T213307_20121120T214521_ADS_000000_5290',
 'DEM1_SAR_DGE_90_20101223T213307_20121120T214521_ADS_000000_8135',
 'DEM1_SAR_DGE_90_20101223T213307_20130327T213545_ADS_000000_2403',
 'DEM1_SAR_DGE_90_20101223T213307_20130327T213545_ADS_000000_6207',
 'DEM1_SAR_DGE_90_20101223T213307_20130327T213545_ADS_000000_9307',
 'DEM1_SAR_DGE_90_20101228T214152_20121012T215300_ADS_000000_5217',
 'DEM1_SAR_DGE_90_20101228T214152_20121114T215252_ADS_000000_2331',
 'DEM1_SAR_DGE_90_20101228T214152_20121114T215252_ADS_000000_3067',
 'DEM1_SAR_DGE_90_20101228T214152_20121114T215252_ADS_000000_6134',
 'DEM1_SAR_DGE_90_20101228T214152_20121114T21525

### Download the DEM tiles into teh `dem` directory

**Note**: the download requires access to CREODIAS services and hence CREODIAS
authentication credentials.
Credentials can be saved into the EODAG configuration file
(default location`~/.config/eodag/eodag.yml`) or via environment variables as
in teh example below.

**CRITICAL**: in addition to `username` and `password`, CREODIAS also requires
a "TOTP" (one time password) that changes every few seconds.
This makes not handy to use EODAG for DEM downloading.

**Note**: archives are automatically extracted.

In [ ]:
%%bash
env EODAG__CREODIAS__AUTH__CREDENTIALS__USERNAME="xxxxx" \
    EODAG__CREODIAS__AUTH__CREDENTIALS__PASSWORD="xxxxx" \
    EODAG__CREODIAS__AUTH__CREDENTIALS__TOTP="xxxxxx" \
eodag download --search-results search_results.geojson --output-dir dem